In [1]:
from selenium import webdriver
import time
import pandas as pd
from datetime import datetime
import random
from tweet_class import Tweet
import re
from collections import deque 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


In [2]:
ACCOUNT_NAME = "RER_A"
SINCE_DATE = "2024-01-01"
#TO_DATE = datetime.now().strftime("%Y-%m-%d")
TO_DATE="2024-06-19"
df = pd.DataFrame(columns=['tweet_type', 'time_posted', 'content', 'associated_tweet_id', 'comments', 'retweets', 'likes', 'views', 'extraction_date'])
NB_ROW = 0
ALREADY_DONE_DATES = set()
#URL= f'https://x.com/search?f=live&q=(from%3A{ACCOUNT_NAME})%20until%3A{TO_DATE}%20since%3A{SINCE_DATE}&src=typed_query'
URL = f'https://x.com/search?f=live&q=(from%3A{ACCOUNT_NAME})%20until%3A{TO_DATE}%20since%3A{SINCE_DATE}%20-filter%3Areplies&src=typed_query'
TWEET_ID_ORDER = deque(maxlen=100)
TWEET_TYPE_ORDER = deque(maxlen=100)

now = datetime.now()
date_string = now.strftime("%Y-%m-%d_%H-%M-%S")
FILENAME = f'test_{date_string}.csv'

In [3]:
options = webdriver.ChromeOptions()
options.add_argument("--mute-audio")
options.add_experimental_option("detach", True)
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')

## Permet de rester connecter
options.add_argument("--user-data-dir=/home/seluser/twitter_scraper/selenium")
options.add_argument("--window-size=900,700")


In [4]:
chrome = webdriver.Chrome(options=options)
chrome.get(URL)

In [5]:
def scrap_tweet(tweet, type):
    tweet_item = Tweet()
    elements = tweet.find_elements('xpath', './div/div/div')
    content = elements[-1].find_elements('xpath', './div[2]/div')

    if len(content)<3:
        print("Tweet with no stats, skip")
        print(len(content))
        return None
    header = content[0]
    content_tweet = content[1]
    stats = content[-1]


        
    all_stats = stats.find_elements('xpath', './div/div/div') 
    try:
        tweet_text_div = content_tweet.find_elements('xpath', './/div[@data-testid="tweetText"]')[0]
        content_tweet = content_tweet.text
    except:
        print("Tweet with no texte, skip")
        content_tweet = ""
    
    try:
        time_posted = header.find_element('xpath', './/time').get_attribute('datetime')
    except Exception:
        time_posted = tweet.find_element('xpath', './/time').get_attribute('datetime')
    time_posted_conv = datetime.strptime(time_posted, "%Y-%m-%dT%H:%M:%S.%fZ")

    get_number_re = re.compile(r'\d+')

    comment = all_stats[0].find_elements('xpath', './button')[0].get_attribute('aria-label')
    if get_number_re.search(comment):
        tweet_item.comment = int(get_number_re.search(comment).group())
    
    retweet = all_stats[1].find_elements('xpath', './button')[0].get_attribute('aria-label')
    if get_number_re.search(retweet):
        tweet_item.retweet = int(get_number_re.search(retweet).group())

    like = all_stats[2].find_elements('xpath', './button')[0].get_attribute('aria-label')
    if get_number_re.search(like):
        tweet_item.like = int(get_number_re.search(like).group())

    views = all_stats[3].find_elements('xpath', './a')[0].get_attribute('aria-label')
    if get_number_re.search(views):
        tweet_item.views = int(get_number_re.search(views).group())



    tweet_item.type = type
    tweet_item.time_posted = time_posted_conv
    tweet_item.content = content_tweet 

    return tweet_item

In [6]:
def scrap_page():
    global NB_ROW
    global ALREADY_DONE_DATES
    global ACCOUNT_NAME
    global FILENAME
    global chrome
    global df
    
    tweets_on_page = chrome.find_elements('xpath', '//article[@data-testid="tweet"]')

    for tweet in tweets_on_page:

        tweet_item = scrap_tweet(tweet, 'Normal')

        if tweet_item is None:
            raise Exception("Tweet is None")
        
        time_posted_conv = tweet_item.time_posted

        if time_posted_conv in ALREADY_DONE_DATES:
                print("Already done")
                continue
        else:
            ALREADY_DONE_DATES.add(time_posted_conv)


        df.loc[NB_ROW] = tweet_item.to_dict()
        NB_ROW += 1

        time.sleep(3)

        url_model = f"/{ACCOUNT_NAME}/status/"
        clickable = tweet.find_element('xpath', f'.//a[contains(@href, "{url_model}")]')
        tweet_url = clickable.get_attribute('href')

        #ActionChains(chrome).key_down(Keys.CONTROL).click(clickable).key_up(Keys.CONTROL).perform()
        chrome.execute_script("window.open(arguments[0], '_blank');", tweet_url)
        chrome.switch_to.window(chrome.window_handles[-1])
        print("Loading")

        WebDriverWait(chrome, 20).until(
            EC.presence_of_all_elements_located((By.XPATH, '//article[@data-testid="tweet"]'))
        )
        time.sleep(5)
        subtweets = chrome.find_elements('xpath', '//article[@data-testid="tweet"]')
        

        #https://x.com/RER_A/status/1818550194557882754

        true_subtweets = subtweets
        try:
            chrome.find_element('xpath', '//span[contains(text(), "Découvrez plus")]')
            print("Il y a Découre Plus")
            true_subtweets = subtweets[1:-1]
        except Exception:
            true_subtweets = subtweets[1:]

        print("Number of subtweets: ", len(true_subtweets))
        for subtweet in true_subtweets:
            elements = subtweet.find_elements('xpath', './div/div/div')
            content = elements[-1].find_elements('xpath', './div[2]/div')
            header = content[0]

            if ACCOUNT_NAME in header.text:
                print("Réponse intéressante")
            elif "Sponsorisé" in subtweet.text:
                print("Sponsorisé")
                continue
            else:
                print('Plus de réponses intéressante...')
                break

            subtweet_item = scrap_tweet(subtweet, 'Réponse')
            if subtweet_item is None:
                raise Exception("Subtweet is None")
        
            df.loc[NB_ROW] = subtweet_item.to_dict()
            NB_ROW += 1



        # Close the new tab
        chrome.close()

        # Switch back to the original tab
        chrome.switch_to.window(chrome.window_handles[-1])

        df.to_csv(FILENAME, index=False)

        



        
        print("---------")

    return list(ALREADY_DONE_DATES)[-1]
    


In [7]:
delay = 3
while True:
    try:
        text_to_search = chrome.find_element('xpath', "//*[contains(text(), 'Essayez de recharger la page.')]")
        spans = chrome.find_elements('xpath', "//*[contains(text(), 'Réessayer')]")
        spans[0].click()
    except:
        pass
    time.sleep(delay + random.randint(1, 3))
    try:
        last_date = scrap_page()
        #last_date = datetime.now() 
        pass
    except Exception as e:
        print("Error while scrapping page")
        print(e)
        continue

    if last_date < datetime.strptime(SINCE_DATE, "%Y-%m-%d"):
        break
    else:
        print("Continue scrolling")
        #chrome.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        chrome.execute_script("window.scrollBy(0, 400);")


Loading
Number of subtweets:  3
Réponse intéressante
Réponse intéressante
Plus de réponses intéressante...
---------
Loading
Number of subtweets:  8
Réponse intéressante
Réponse intéressante
Réponse intéressante
Plus de réponses intéressante...
---------
Loading
Number of subtweets:  1
Réponse intéressante
---------
Loading
Number of subtweets:  10
Plus de réponses intéressante...
---------
Loading
Number of subtweets:  1
Réponse intéressante
---------
Loading
Number of subtweets:  7
Réponse intéressante
Réponse intéressante
Réponse intéressante
Plus de réponses intéressante...
---------
Loading
Number of subtweets:  8
Réponse intéressante
Plus de réponses intéressante...
---------
Loading
Number of subtweets:  1
Réponse intéressante
---------
Loading
Number of subtweets:  1
Réponse intéressante
---------
Loading
Number of subtweets:  2
Plus de réponses intéressante...
---------
Loading
Number of subtweets:  0
---------
Loading
Number of subtweets:  2
Réponse intéressante
Plus de répon

KeyboardInterrupt: 

: 